# DECISION TREE Y RANDOM FOREST
---

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

In [3]:
df_est = pd.read_csv("datos/05-bikes_encoding_est.csv", index_col = 0)
df_est.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,registered
0,0,0,0,0,0,1,2,14.110847,-0.680818,1.252343,-0.387833,654
1,0,0,0,7,2,2,2,14.902598,-0.741507,0.480996,0.748899,670
2,0,0,1,7,2,2,4,8.050924,-1.750344,-1.338073,0.745931,1229
3,1,0,1,7,1,1,4,8.200000,-1.610886,-0.261577,-0.389769,1454
4,1,0,2,7,1,2,4,9.305237,-1.505615,-1.340294,-0.046477,1518


In [4]:
df_est.season.unique()

array([0, 1, 2])

In [5]:
df_est.isnull().sum()

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
registered    0
dtype: int64

In [6]:
# Creamos una clase para poder hacer el decision tree y el random forest más rápido
class Regres_lineal:
    def __init__(self, dataf, col_pred):
        
        self.dataf = dataf
        self.col_pred = col_pred

    def tree_param(self):
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        arbol = DecisionTreeRegressor(random_state =0)
    
        arbol.fit(x_train, y_train)

        max_feat = np.sqrt(len(x_train.columns))
        max_dep = arbol.tree_.max_depth
        return(max_feat, max_dep)

    def metricas(self, y_te, y_tr, y_te_pred, y_tr_pred, tipo_modelo):
        resultados = {'MAE': [mean_absolute_error(y_te, y_te_pred), mean_absolute_error(y_tr, y_tr_pred)],
                    'MSE': [mean_squared_error(y_te, y_te_pred), mean_squared_error(y_tr, y_tr_pred)],
                    'RMSE': [np.sqrt(mean_squared_error(y_te, y_te_pred)), np.sqrt(mean_squared_error(y_tr, y_tr_pred))],
                    'R2':  [r2_score(y_te, y_te_pred), r2_score(y_tr, y_tr_pred)],
                    "set": ["test", "train"]}
        df = pd.DataFrame(resultados)
        df["modelo"] = tipo_modelo
        return df

    def grid_s_decision(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= GradientBoostingRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

    def grid_s_forest(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= RandomForestRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

In [7]:

rg_est = Regres_lineal(df_est, "registered")

In [8]:
features_est, depth_est = rg_est.tree_param()

In [9]:
features_est #El número máx será 3

3.3166247903554

In [10]:
depth_est # La profundidad máxima será 20

20

In [11]:
param1 = {"max_depth": [16, 18, 20], 
        "max_features": [3,4],
        "min_samples_split": [50, 100],
        "min_samples_leaf": [50,100]} 

In [12]:
best_model1, result1 = rg_est.grid_s_decision(param1, "GradientBoosting Estand I")

In [13]:
best_model1

GradientBoostingRegressor(max_depth=16, max_features=3, min_samples_leaf=50,
                          min_samples_split=100)

In [14]:
result1

,MAE,MSE,RMSE,R2,set,modelo
0,520.130212,440709.535173,663.859575,0.795833,test,GradientBoosting Estand I
1,485.976752,394649.346055,628.211227,0.841702,train,GradientBoosting Estand I


In [15]:
param2 = {"max_depth": [4,6,8,10, 12, 14, 16, 18, 20], 
        "max_features": [2, 3,4],
        "min_samples_split": [10, 20, 40, 50, 60],
        "min_samples_leaf": [10, 20, 40, 50, 60]} 

In [16]:
best_model2, result2 = rg_est.grid_s_decision(param2, "GradientBoosting Estand II")

In [17]:
best_model2

GradientBoostingRegressor(max_depth=12, max_features=2, min_samples_leaf=10,
                          min_samples_split=40)

In [18]:
result2

,MAE,MSE,RMSE,R2,set,modelo
0,564.135728,488815.239564,699.153230,0.773547,test,GradientBoosting Estand II
1,272.246865,132281.616555,363.705398,0.946941,train,GradientBoosting Estand II


In [19]:
param3 = {"max_depth": [6, 8, 10, 12, 14, 16, 18], 
        "max_features": [3,4],
        "min_samples_split": [10, 20, 40],
        "min_samples_leaf": [10, 20, 40]} 

In [20]:
best_model3, result3 = rg_est.grid_s_forest(param3, "RandForest Estand I")

In [21]:
best_model3

RandomForestRegressor(max_depth=14, max_features=4, min_samples_leaf=10,
                      min_samples_split=10)

In [22]:
result3

,MAE,MSE,RMSE,R2,set,modelo
0,574.810179,545135.537647,738.332945,0.747455,test,RandForest Estand I
1,556.257489,510822.212417,714.718275,0.795104,train,RandForest Estand I


In [23]:
param4 = {"max_depth": [12, 14, 16, 18], 
        "max_features": [4,5],
        "min_samples_split": [10, 20, 40, 50],
        "min_samples_leaf": [10, 20, 40, 50]} 

In [24]:
best_model4, result4 = rg_est.grid_s_forest(param4, "RandForest Estand II")

In [25]:
best_model4

RandomForestRegressor(max_depth=12, max_features=5, min_samples_leaf=10,
                      min_samples_split=20)

In [26]:
result4

,MAE,MSE,RMSE,R2,set,modelo
0,567.836731,532889.748010,729.992978,0.753128,test,RandForest Estand II
1,548.127824,501232.279126,707.977598,0.798951,train,RandForest Estand II


In [27]:
df_sinest = pd.read_csv("datos/04-bikes_encoding_sinest.csv", index_col = 0)
df_sinest.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,registered
0,0,0,0,0,0,1,2,14.110847,18.18125,80.5833,10.749882,654
1,0,0,0,7,2,2,2,14.902598,17.68695,69.6087,16.652113,670
2,0,0,1,7,2,2,4,8.050924,9.47025,43.7273,16.636703,1229
3,1,0,1,7,1,1,4,8.200000,10.60610,59.0435,10.739832,1454
4,1,0,2,7,1,2,4,9.305237,11.46350,43.6957,12.522300,1518


In [28]:
rg_sinest = Regres_lineal(df_sinest, "registered")

In [29]:
features_sinest, depth_sinest = rg_sinest.tree_param()

In [30]:
features_sinest #El número máx será 3

3.3166247903554

In [31]:
depth_sinest # La profundidad máxima será 22

20

In [32]:
best_model5, result5 = rg_sinest.grid_s_decision(param1, "GradientBoosting SinEstand I")

In [33]:
result5

,MAE,MSE,RMSE,R2,set,modelo
0,513.621287,417208.026234,645.916424,0.806720,test,GradientBoosting SinEstand I
1,470.065509,371916.134159,609.849272,0.850821,train,GradientBoosting SinEstand I


In [34]:
best_model6, result6 = rg_sinest.grid_s_decision(param2, "GradientBoosting SinEstand II")

In [35]:
result6

,MAE,MSE,RMSE,R2,set,modelo
0,557.819572,479395.317983,692.383794,0.777911,test,GradientBoosting SinEstand II
1,435.223461,321699.328954,567.185445,0.870963,train,GradientBoosting SinEstand II


In [36]:
best_model7, result7 = rg_sinest.grid_s_forest(param3, "RandForest SinEstand I")

In [37]:
result7

,MAE,MSE,RMSE,R2,set,modelo
0,583.315761,542001.829995,736.207736,0.748907,test,RandForest SinEstand I
1,561.562531,511509.048288,715.198608,0.794829,train,RandForest SinEstand I


In [38]:
best_model8, result8 = rg_sinest.grid_s_forest(param4, "RandForest SinEstand II")

In [39]:
result8

,MAE,MSE,RMSE,R2,set,modelo
0,567.604696,534769.058554,731.279057,0.752258,test,RandForest SinEstand II
1,551.047542,504653.952022,710.390000,0.797578,train,RandForest SinEstand II


In [40]:
param5 = {"max_depth": [22], 
        "max_features": [4],
        "min_samples_split": [8],
        "min_samples_leaf": [8]}

In [41]:
best_model9, result9 = rg_sinest.grid_s_forest(param5, "RandForest SinEstand III")

In [42]:
result9

,MAE,MSE,RMSE,R2,set,modelo
0,575.296401,553373.737883,743.890945,0.743639,test,RandForest SinEstand III
1,533.316633,474729.947854,689.006493,0.809581,train,RandForest SinEstand III


In [43]:
best_model10, result10 = rg_est.grid_s_forest(param5, "RandForest Estand III")

In [44]:
result10

,MAE,MSE,RMSE,R2,set,modelo
0,573.478772,535201.445882,731.574635,0.752058,test,RandForest Estand III
1,531.269120,464139.629623,681.277939,0.813829,train,RandForest Estand III


In [45]:
param6 = {"max_depth": [18,20], 
        "max_features": [3,4],
        "min_samples_split": [8, 15],
        "min_samples_leaf": [8, 15]}

In [46]:
best_model11, result11 = rg_est.grid_s_forest(param6, "RandForest Estand IV")

In [47]:
best_model11

RandomForestRegressor(max_depth=18, max_features=4, min_samples_leaf=8,
                      min_samples_split=15)

In [48]:
result11

,MAE,MSE,RMSE,R2,set,modelo
0,571.512062,534750.746715,731.266536,0.752266,test,RandForest Estand IV
1,535.800815,474831.587490,689.080247,0.809540,train,RandForest Estand IV


In [49]:
best_model12, result12 = rg_est.grid_s_decision(param5, "GradientBoosting Estand III")

In [50]:
result12

,MAE,MSE,RMSE,R2,set,modelo
0,562.255296,516231.293577,718.492375,0.760846,test,GradientBoosting Estand III
1,93.209842,18497.664083,136.006118,0.992580,train,GradientBoosting Estand III


In [51]:
best_model13, result13 = rg_sinest.grid_s_decision(param5, "GradientBoosting SinEstand III")

In [57]:
best_model13

GradientBoostingRegressor(max_depth=22, max_features=4, min_samples_leaf=8,
                          min_samples_split=8)

In [52]:
result13

,MAE,MSE,RMSE,R2,set,modelo
0,599.334873,558419.369447,747.274628,0.741301,test,GradientBoosting SinEstand III
1,93.354286,18124.091512,134.625746,0.992730,train,GradientBoosting SinEstand III


In [53]:
param7 = {"max_depth": [18,20], 
        "max_features": [4],
        "min_samples_split": [8, 16],
        "min_samples_leaf": [8]}

In [54]:
best_model14, result14 = rg_est.grid_s_decision(param7, "GradientBoosting Estand IV")

In [55]:
best_model14

GradientBoostingRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                          min_samples_split=16)

In [56]:
result14

,MAE,MSE,RMSE,R2,set,modelo
0,576.609113,540637.882481,735.280819,0.749539,test,GradientBoosting Estand IV
1,93.563020,18587.956781,136.337657,0.992544,train,GradientBoosting Estand IV


In [58]:
param17 = {"max_depth": [10], 
        "max_features": [3],
        "min_samples_split": [50],
        "min_samples_leaf": [10]}

best_model15, result15 = rg_est.grid_s_decision(param17, "GradientBoosting Estand IV")

result15

,MAE,MSE,RMSE,R2,set,modelo
0,536.244885,452433.640667,672.631876,0.790401,test,GradientBoosting Estand IV
1,270.996421,129663.300792,360.087907,0.947991,train,GradientBoosting Estand IV


In [78]:
param17 = {"max_depth": [16], 
        "max_features": [4],
        "min_samples_split": [40],
        "min_samples_leaf": [20]}

best_model16, result16 = rg_sinest.grid_s_decision(param17, "GradientBoosting SinEstand IV")

result16

,MAE,MSE,RMSE,R2,set,modelo
0,519.469836,443119.926705,665.672537,0.794716,test,GradientBoosting SinEstand IV
1,290.322784,152409.720458,390.396876,0.938867,train,GradientBoosting SinEstand IV
